## 训练过程中将callbacks作用于模型

当观测到验证损失不再改善时就停止训练。这可以使用`callbacks`来实现。

**callbacks**是在调用`fit`方法时传入模型的一个对象（即实现特定方法的类实例），它在训练过程中的不同时间点都会被模型调用。它可以访问关于模型状态和性能的所有可用数据，还可以采取行动：中断训练、保存模型、加载一组不同的权重或改变模型的状态。

### `ModelCheckpoint`与`EarlyStopping` callbacks

如果监控的目标指标在设定的`epochs`内不再改善，可以用`EarlyStopping` callback来中断训练。这个函数通常与`ModelCheckpoint`结合使用，后者可以在训练过程中持续不断地保存模型。

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.2'

```python
callbacks_list = [  # 模型不再改善，就中断训练
    keras.callbacks.EarlyStopping(monitor='val_acc',  # 监控模型的验证精度
                                  patience=1,  # 如果精度在多于1轮的时间内不再改善，中断训练
                                  verbose=0,
                                  mode='auto'),
    keras.callbacks.ModelCheckpoint(filepath='my_model.h5',  # 在每轮过后保存当前权重
                                    monitor='val_loss',
                                    save_best_only=True,
                                    save_weights_only=False,  # 若为True，只保存模型权重
                                    mode='auto',
                                    period=1)]   # Checkpint之间间隔的epoch数

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])  # 监控accuracy，所以它应该是模型指标的一部分

# 因为callback要监控验证精度和验证损失，所以在调用`fit`时需要传入`validation_data`
model.fit(X, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data(X_val, y_val))
```

### `ReduceLROnPlateau` callback

如果验证损失不再改善，可以使用这个callback来降低学习率。在训练过程中如果出现了**损失平台（loss plateau）**，那么增大或减小学习率都是跳出局部最小值的有效策略。

```python
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',  # 监控模型的验证损失
                                      factor=0.1,  # 触发时将学习率除以10
                                      patience=10)  # 如果验证损失在10轮内后都没有改善，那么就触发这个callback
]

# 因为callback要监控验证精度和验证损失，所以在调用`fit`时需要传入`validation_data`
model.fit(X, y,
          epochs=10,
          batch_size=32
          callbacks=callbacks_list,
          validation_data=(X_val, y_val))
```